# Saving Loess Detrended Data
Saving all data with loess detrend.

<b> PHD-1 </b>
* Switched read_and_merge_netcdfs from longrunmip_loaded function to open_ds - all opening files are now done within the one py file
* longrunmip loader removed. Better just to load files in one by one
* Moved from init to top level
* Removed all extra code (see nb 23 in init for old code).


In [14]:
import xarray as xr
from importlib import reload
import numpy as np
import pandas as pd
import os
from glob import glob
from time import perf_counter

In [15]:
import sys
sys.path.append('modules')
import constants
import xarray_class_accessors as xca
import open_ds
import signal_to_noise as sn

In [16]:
from multiprocessing import Pool
from functools import partial

In [17]:
chunks = {'lat':48/2,'lon':96/2,'time':-1}

# Opening All Data

In [18]:
open_kwargs = dict(use_cftime=True, concat_dim='model', combine='nested', 
                  chunks = {'lat': 100, 'lon': 100, 'time': -1})

In [19]:
os.listdir(os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR, '4xCO2'))

['CanESM5.nc',
 'UKESM1-0-LL.nc',
 'NorESM2-LM.nc',
 'GFDL-ESM4.nc',
 'MPI-ESM1-2-LR.nc',
 'CESM2.nc',
 'MIROC-ES2L.nc',
 'GISS-E2-1-G-CC.nc']

In [20]:
experiment_ds = xr.open_mfdataset(os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR, '4xCO2', '*.nc'), 
                                 use_cftime=True).compute()
experiment_ds = experiment_ds.isel(time=slice(0,100))
experiment_ds

<xarray.Dataset>
Dimensions:  (time: 100, model: 8, lat: 96, lon: 192)
Coordinates:
  * time     (time) object 0001-12-31 00:00:00 ... 0100-12-31 00:00:00
  * model    (model) object 'CESM2' 'CanESM5' ... 'NorESM2-LM' 'UKESM1-0-LL'
    height   float64 2.0
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 83.0 84.86 86.72 88.57
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
Data variables:
    tas      (time, model, lat, lon) float32 228.2 228.1 228.0 ... 269.3 269.4

In [21]:
picontrol_ds = xr.open_mfdataset(os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR, 'picontrol', '*.nc'), 
                  use_cftime=True, concat_dim='model', combine='nested').compute()
picontrol_ds

<xarray.Dataset>
Dimensions:  (time: 1200, model: 8, lat: 96, lon: 192)
Coordinates:
  * time     (time) object 0001-12-31 00:00:00 ... 1200-12-31 00:00:00
  * model    (model) object 'CESM2' 'CanESM5' ... 'NorESM2-LM' 'UKESM1-0-LL'
    height   float64 2.0
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 83.0 84.86 86.72 88.57
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
Data variables:
    tas      (time, model, lat, lon) float32 225.1 225.0 224.9 ... nan nan nan

In [22]:
# Calculating the anomalies
experiment_anom_ds = experiment_ds.clima_ds.anomalies(picontrol_ds)
# Space mean of anomalies
experiment_anom_ds = experiment_anom_ds.compute()
experiment_anom_ds

tas, Using historical dataset


<xarray.Dataset>
Dimensions:  (time: 100, model: 8, lat: 96, lon: 192)
Coordinates:
  * time     (time) object 0001-12-31 00:00:00 ... 0100-12-31 00:00:00
  * model    (model) object 'CESM2' 'CanESM5' ... 'NorESM2-LM' 'UKESM1-0-LL'
    height   float64 2.0
  * lat      (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 83.0 84.86 86.72 88.57
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
Data variables:
    tas      (time, model, lat, lon) float32 2.608 2.6 2.592 ... 16.5 16.59

In [28]:
def loess_function(window, da, history, save_name, directory):
    '''
    Function to use for saving of less filtered data.
    '''
    print(window)

    # Run the loess grid
    loess_ds = da.sn.loess_grid(step_size=window)
    
    # Add history
    loess_ds.attrs = history
    
    
    # Save the file
    file_save_path = os.path.join(directory, f'{save_name}_{window}.nc')
    print(f'Saving {file_save_path}')
    loess_ds.to_netcdf(file_save_path)

In [29]:
# Adding some attributes
now = pd.Timestamp.now().strftime(format = '%Y/%m/%d')
history = {'history': f'Modified by Alex Borowiak on {now}','stats': 'loess detrended data'}
history

{'history': 'Modified by Alex Borowiak on 2022/08/24',
 'stats': 'loess detrended data'}

In [30]:
experiment_loess_function = partial(loess_function, da=experiment_anom_ds.tas, history=history,
                             directory = os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR,  'loess'),
                                   save_name = '1pct_CO2_branch')

picontrol_loess_function = partial(loess_function, da=picontrol_ds.tas, history=history,
                             directory = os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR,  'loess'),
                                     save_name = 'control')

In [31]:
%%time
window_values = np.arange(10, 30, 5)
with Pool() as pool:
    pool.map(experiment_loess_function, window_values)

10
15
20
25
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_10.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_20.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_15.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_25.nc
10
15
20
25
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_10.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_15.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_25.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/1pct_CO2_branch_20.nc
CPU times: user 664 ms, sys: 1.04 s, total: 1.7 s
Wall time: 1h 21min 21s


In [32]:
%%time
window_values = np.arange(10, 30, 5)
with Pool() as pool:
    pool.map(picontrol_loess_function, window_values)

10
15
20
25
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/control_10.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/control_15.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/control_20.nc
Saving /g/data/w40/ab2313/PhD/zecmip/regridded/loess/control_25.nc
CPU times: user 3.08 s, sys: 7.71 s, total: 10.8 s
Wall time: 1h 5min 32s


In [ ]:
# for window in np.arange(10, 30, 5):
#     print(window)

#     loess_experiment_anom_ds = experiment_anom_ds.tas.sn.loess_grid(step_size=window)

#     loess_control_ds = picontrol_ds.tas.sn.loess_grid(step_size=window)

#     # Adding some attributes
#     now = pd.Timestamp.now().strftime(format = '%Y/%m/%d')
#     history = {'history': f'Modified by Alex Borowiak on {now}','stats': 'loess detrended data'}
#     loess_experiment_anom_ds.attrs = history
#     loess_control_ds.attrs = history

#     loess_experiment_anom_ds.to_netcdf(
#         os.path.join(constants.ZECMIP_G_DIR,  'loess', f'1pct_branch_{window}.nc'))
    
#     loess_control_ds.to_netcdf(
#         os.path.join(constants.ZECMIP_G_DIR, 'loess', f'picontrol_{window}.nc'))